In [20]:
import numpy as np
import pandas as pd

# Set seed for reproducibility
np.random.seed(2)

# Define number of states and actions
n_states = 3
n_actions = 3

# Generate transition probability matrices P[a][s][s']
P = np.zeros((n_actions, n_states, n_states))
for a in range(n_actions):
    for s in range(n_states):
        row = np.random.rand(n_states)
        P[a][s] = row / row.sum()  # normalize to make it a valid probability distribution

# Transition matrices P[0], P[1], P[2]
P_0 = P[0]
P_1 = P[1]
P_2 = P[2]

# Generate reward matrix R[s][a], values in [0, 2]
R = np.random.uniform(0, 2, size=(n_states, n_actions))

print("Transition Probability Matrices:")
print("P[accion=0]:\n", P_0)
print("")
print("P[accion=1]:\n", P_1)
print("")
print("P[accion=2]:\n", P_2)
print("")

print("\nReward Matrix R:")
print(R)

Transition Probability Matrices:
P[accion=0]:
 [[0.43100234 0.02562935 0.54336831]
 [0.36704318 0.35443418 0.27852264]
 [0.18214073 0.55116157 0.2666977 ]]

P[accion=1]:
 [[0.18829064 0.43831235 0.37339701]
 [0.16163858 0.61683809 0.22152333]
 [0.36808894 0.40026078 0.23165029]]

P[accion=2]:
 [[0.5914001  0.0556396  0.35296029]
 [0.11066639 0.72570517 0.16362843]
 [0.13386426 0.62820766 0.23792809]]


Reward Matrix R:
[[0.21389137 0.44061241 0.69965257]
 [0.93557497 0.40348645 1.28081345]
 [0.96613967 1.01047344 0.7737853 ]]


In [32]:

# This code implements a simple Q-learning algorithm for a Markov Decision Process (MDP)
# with randomly generated transition probabilities and rewards.

# ---- Q-learning Parameters ----
alpha = 0.1       # learning rate
gamma = 0.95      # discount factor
epsilon = 0.1    # exploration rate
episodes = 10000
max_steps = 500    # per episode

# ---- Initialize Q-table ----
Q = np.random.rand(n_states, n_actions)

# ---- Q-learning Loop ----
for episode in range(episodes):
    s = np.random.randint(0, n_states)  # Start in random state

    for _ in range(max_steps):
        # ε-greedy action selection
        if np.random.rand() < epsilon:
            a = np.random.randint(0, n_actions)  # Explore
        else:
            a = np.argmax(Q[s])  # Exploit

        # Sample next state s' from P[a][s]
        s_prime = np.random.choice(n_states, p=P[a][s])
        r = R[s, a]

        # Q-value update
        Q[s, a] += alpha * (r + gamma * np.max(Q[s_prime]) - Q[s, a])

        s = s_prime  # Move to next state

# ---- Output the learned Q* ----
print("Learned Q* Table:")
for i in range(n_states):
    for j in range(n_actions):
        print(f"Q({i},{j}) = {Q[i,j]:.4f}", end="\t")
    print()


Learned Q* Table:
Q(0,0) = 20.5011	Q(0,1) = 20.8991	Q(0,2) = 20.8529	
Q(1,0) = 21.2468	Q(1,1) = 21.1205	Q(1,2) = 21.7352	
Q(2,0) = 21.4309	Q(2,1) = 21.3494	Q(2,2) = 21.3218	


In [35]:
# --- PARAMETERS ---
gamma = 0.95       # discount factor
s = 1              # state index
a = 1              # action index

# --- Bellman Evaluation ---
# Get reward
r_sa = R[s, a]

# Get transition probabilities for taking action a in state s
p_s_prime = P[a][s]  # array of P(s' | s, a)

# Compute max_a' Q(s', a') for each possible s'
max_q_s_prime = np.max(Q, axis=1)  # array of size [n_states]

# Compute expected future return: ∑ P(s'|s,a) * max_a' Q(s',a')
expected_future_value = np.dot(p_s_prime, max_q_s_prime)

# Bellman right-hand side
rhs = r_sa + gamma * expected_future_value

# Q* value (left-hand side)
lhs = Q[s, a]

# --- PRINT RESULTS ---
print(f"Checking Bellman equation for Q({s}, {a}):")
print(f"  Q*({s},{a})        = {lhs:.6f}")
print(f"  R({s},{a})         = {r_sa:.6f}")
print(f"  Expected max Q(s') = {expected_future_value:.6f}")
print(f"  RHS (Bellman)      = {rhs:.6f}")
print(f"  Difference         = {lhs - rhs:.6f}   as % of Q* = {(lhs - rhs) / lhs * 100:.2f}%")


Checking Bellman equation for Q(1, 1):
  Q*(1,1)        = 21.120473
  R(1,1)         = 0.403486
  Expected max Q(s') = 21.532680
  RHS (Bellman)      = 20.859532
  Difference         = 0.260940   as % of Q* = 1.24%
